<a href="https://colab.research.google.com/github/sravani1102/AI/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Find-S**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/Finds1.xlsx')
df

,Colour,Toughness,Fungus,Apperance,Poisonous
0,Green,Hard,NO,W,YES
1,Green,Hard,YES,S,NO
2,Brown,Soft,NO,W,NO
3,Orange,Hard,NO,W,YES
4,Green,Soft,YES,S,YES
5,Green,Hard,YES,W,YES
6,Orange,Hard,NO,W,YES


In [ ]:
df=df[df['Poisonous']=='YES']
df

,Colour,Toughness,Fungus,Apperance,Poisonous
0,Green,Hard,NO,W,YES
3,Orange,Hard,NO,W,YES
4,Green,Soft,YES,S,YES
5,Green,Hard,YES,W,YES
6,Orange,Hard,NO,W,YES


In [ ]:
def findS(df):
  h=['pi']*(len(df.columns)-1)
  d=np.array(df)
  for i in d:
    if 'pi' in h:
      for j in range(len(h)):
        h[j]=i[j]
    else:
      for j in range(len(h)):
        if h[j]!=i[j]:
          h[j]='?'
  print(h)

In [ ]:
findS(df)

['?', '?', '?', '?']


**Candidate Elimination**

In [77]:
import pandas as pd
import numpy as np
df=pd.read_excel('/content/drive/MyDrive/candiadate.xlsx')
df

,Sky,AirTemp,Humidity,Wind,Water,Forecast,EnjoySport
0,Sunny,Warm,Normal,Strong,Warm,Same,YES
1,Sunny,Warm,High,Strong,Warm,Same,YES
2,Rainy,Cold,High,Strong,Warm,Change,NO
3,Sunny,Warm,High,Strong,Cool,Change,YES


In [78]:
def specified(s,p):
  if 'pi' in s:
    for j in range(len(s)):
      s[j]=p[j]
  else:
    for j in range(len(s)):
      if s[j]!=p[j]:
        s[j]='?'
  return s

In [79]:
def accept(l,df):
  for j in range(len(df)):
    if df.iloc[j,-1]=='YES':
      for k in range(len(l)):
        if l[k]!='?' and df.iloc[j,k]!=l[k]:
          return 0
    else:
      c=0
      for k in range(len(l)):
        if l[k]=='?' or df.iloc[j,k]==l[k]:
          c=c+1
      if c==len(l):
        return 0
  return 1

In [84]:
def generalised(df,i):
  g1=[]
  for j in range(len(df.columns)-1):
    m=list(np.unique(df[df.columns[j]]))
    m.remove(df.iloc[i,j])
    l=['?']*(len(df.columns)-1)
    for k in m:
      l[j]=k
      if accept(l,df):
        g1.append(l)
  return g1

In [85]:
def learned(s,df,g):
  lvs=[]
  for i in range(len(g)):
    for j in range(len(s)):
      l=g[i].copy()
      if g[i][j]=='?' and s[j]!='?':
          l[j]=s[j]
          if accept(l,df) and l not in g and l not in lvs:
            lvs.append(l)
  return lvs

In [86]:
def candidate(df):
  s=['pi']*(len(df.columns)-1)
  g=[]
  for i in range(len(df)):
    if df.iloc[i,-1]=='YES':
      s=specified(s,df.iloc[i])
    else:
      g.extend(generalised(df,i))
  if len(g)!=0:
    lvs=learned(s,df,g)
    print(s,'\n',g,'\n',lvs)
  else:
    print(s)

In [87]:
candidate(df)

['Sunny', 'Warm', '?', 'Strong', '?', '?'] 
 [['Sunny', '?', '?', '?', '?', '?'], ['?', 'Warm', '?', '?', '?', '?']] 
 [['Sunny', 'Warm', '?', '?', '?', '?'], ['Sunny', '?', '?', 'Strong', '?', '?'], ['?', 'Warm', '?', 'Strong', '?', '?']]


**Navies Bayes**

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/aimldatasets/tabletb.csv')
df=df.iloc[:,1:-1]
df

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [ ]:
yes=df['Play Tennis'][df['Play Tennis']=="Yes"].count()
no=df['Play Tennis'][df['Play Tennis']=="No"].count()

In [ ]:
def navieBayes(df):
  p={}
  for x in df.columns[:-1]:
    l=df[x].unique()
    y=n=0
    for z in l:
      k=df[df[x]==z][df.columns[-1]]
      y=k[k=="Yes"].count()/yes
      n=k[k=="No"].count()/no
      p.update({z:[y,n]})
  return p


In [ ]:
p=navieBayes(df)
p

{'Sunny': [0.2222222222222222, 0.6],
 'Overcast': [0.4444444444444444, 0.0],
 'Rain': [0.3333333333333333, 0.4],
 'Hot': [0.2222222222222222, 0.4],
 'Mild': [0.4444444444444444, 0.4],
 'Cool': [0.3333333333333333, 0.2],
 'High': [0.3333333333333333, 0.8],
 'Normal': [0.6666666666666666, 0.2],
 'Weak': [0.6666666666666666, 0.4],
 'Strong': [0.3333333333333333, 0.6]}

In [ ]:
def test(l,p):
  y=yes
  n=no
  for j in l:
    y*=p[j][0]
    n*=p[j][1]
  if y>n:
    print("Yes")
  else:
    print("No")

In [ ]:
test(['Rain','Hot','High','Weak'],p)

No


**Navies Bayes Accuracy**

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/aimldatasets/tabletb.csv')
df=df.iloc[:,1:-1]
df

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [ ]:
x_train=df[:9]
x_test=df.iloc[9:]
#y_test=df[9:][df[9:].columns[-1]]
yes_train=x_train['Play Tennis'][x_train['Play Tennis']=="Yes"].count()
no_train=x_train['Play Tennis'][x_train['Play Tennis']=="No"].count()
yes=yes_train/len(x_train)
no=no_train/len(x_train)

In [ ]:
x_train

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes


In [ ]:
def probabilties(x_train):
  p_train={}
  for x in x_train.columns[:-1]:
    l=x_train[x].unique()
    y=n=0
    for z in l:
      k=x_train[x_train[x]==z][x_train.columns[-1]]
      y=k[k=="Yes"].count()/yes_train
      n=k[k=="No"].count()/no_train
      p_train.update({z:[y,n]})
  return p_train

In [ ]:
p_train=probabilties(x_train)
p_train

{'Sunny': [0.2, 0.75],
 'Overcast': [0.4, 0.0],
 'Rain': [0.4, 0.25],
 'Hot': [0.2, 0.5],
 'Mild': [0.2, 0.25],
 'Cool': [0.6, 0.25],
 'High': [0.4, 0.75],
 'Normal': [0.6, 0.25],
 'Weak': [0.8, 0.5],
 'Strong': [0.2, 0.5]}

In [ ]:
x_test

,Outlook,Temperature,Humidity,Wind,Play Tennis
9,Rain,Mild,Normal,Weak,Yes
10,Sunny,Mild,Normal,Strong,Yes
11,Overcast,Mild,High,Strong,Yes
12,Overcast,Hot,Normal,Weak,Yes
13,Rain,Mild,High,Strong,No


In [ ]:
def test(x_test,p_train):
  classify=[]
  for i in range(len(x_test)):
    y=yes
    n=no
    for j in (x_test.columns[:-1]):
      y*=p_train[x_test.iloc[i][j]][0]
      n*=p_train[x_test.iloc[i][j]][1]
    if y>n:
      classify.append("Yes")
    else:
      classify.append("No")
  return classify

In [ ]:
actual=list(x_test.iloc[:,-1])
predicted=test(x_test,p_train)
print(actual,'\n',predicted)

['Yes', 'Yes', 'Yes', 'Yes', 'No'] 
 ['Yes', 'No', 'Yes', 'Yes', 'No']


In [ ]:
from sklearn.metrics import accuracy_score
z=accuracy_score(actual, predicted)
print("Accuracy = ",100*z)

Accuracy =  80.0


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(actual, predicted)

array([[1, 0],
       [1, 3]])

**Text Classification**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/aimldatasets/text_classification - text_classification.csv")
df['Text']=df['Text'].str.lower()
df

,Text,Label
0,i love this sandwich,pos
1,this is an amazing place,pos
2,i feel very good about these beers,pos
3,this is my best work,pos
4,what an awesome view,pos
5,i do not like this restaurant,neg
6,i am tired of this stuff,neg
7,i can't deal with this,neg
8,he is my sworn enemy,neg
9,my boss is horrible,neg


In [ ]:
text_train=df[:10]
text_test=df[10:]

In [ ]:
def train(text):
  vocab=[]
  pos=[]
  neg=[]
  for i in range(len(text)):
    s=text.iloc[i,:-1].tolist()
    l=s[0].split(" ")
    vocab.extend(l)
    if(text.iloc[i,-1]=="pos"):
      pos.extend(l)
    else:
      neg.extend(l)
  vocab=list(set(vocab))
  return pos,neg,vocab

In [ ]:
def classify(test,tp,tn,p,n,vocab):
  classify=[]
  for i in range(len(test)):
    pos=tp
    neg=tn
    s=test[i].split(" ")
    for j in s:
      if j in vocab:
        pos*=(p.count(j)+1)/(len(p)+len(vocab))
        neg*=(n.count(j)+1)/(len(n)+len(vocab))
      else:
        pos*=1/(len(p)+len(vocab))
        neg*=1/(len(n)+len(vocab))
    if pos>=neg:
      classify.append("pos")
    else:
      classify.append("neg")
  return classify

In [ ]:
p=text_train.iloc[:,-1]
tp=p[p=="pos"].count()/len(p)
tn=p[p=="neg"].count()/len(p)
pos,neg,vocab=train(text_train)
actual=text_test['Label'].tolist()
predicted=classify(text_test['Text'].tolist(),tp,tn,pos,neg,vocab)
print(actual,"\n",predicted)

['pos', 'neg', 'pos', 'neg', 'pos', 'neg', 'pos', 'neg'] 
 ['pos', 'neg', 'pos', 'neg', 'pos', 'pos', 'pos', 'neg']


In [ ]:
from sklearn.metrics import accuracy_score
z=accuracy_score(actual, predicted)


Accuracy =  87.5


confusion matrix :
[[True Negative,False Positive],[False Negative,True Positive]]

In [ ]:
from sklearn.metrics import confusion_matrix
c=confusion_matrix(actual, predicted)
print(c)

[[3 1]
 [0 4]]


In [ ]:
precision=c[1][1]/(c[1][1]+c[0][1])
recall=c[1][1]/(c[1][1]+c[1][0])
print("Accuracy = ",100*z)
print("Precision :",precision)
print("Recall :",recall)

Accuracy =  87.5
Precision : 0.8
Recall : 1.0


**Bayesian**

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/aimldatasets/heart.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


**K-Nearset Negihbours**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/aimldatasets/Iris.csv")
df=df.iloc[:,1:]
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
x_train=df[:130]
x_test=df[130:]

In [ ]:
from math import sqrt
def euclideanDistance(iris,l):
  p=[]
  for i in range(len(iris)):
    dist=0
    for j in range(len(iris.columns)):
      dist+=(iris.iloc[i,j]-l[j])**2
    p.append(sqrt(dist))
  return p

In [ ]:
def Rank(dist):
  k=sorted(dist)
  return [dist.index(k[0]),dist.index(k[1]),dist.index(k[2])]

In [ ]:
def knn(x_train,l):
  dist=euclideanDistance(x_train.iloc[:,:-1],l)
  rank=Rank(dist)
  species=pd.unique(x_train.iloc[:,-1])
  max=0
  for x in species:
    delta=0
    for y in rank:
      if df.iloc[y,-1]==x:
        delta+=1
    if delta>max:
      max=delta
      c=x
  return c

In [ ]:
classify=[]
test=x_test.iloc[:,:-1].to_numpy()
for i in range(len(x_test)):
  classify.append(knn(x_train,test[i]))
classifier=pd.DataFrame({"Actual":x_test.iloc[:,-1].tolist(),"Predicted":classify})
print(classifier)

            Actual        Predicted
0   Iris-virginica   Iris-virginica
1   Iris-virginica   Iris-virginica
2   Iris-virginica   Iris-virginica
3   Iris-virginica  Iris-versicolor
4   Iris-virginica   Iris-virginica
5   Iris-virginica   Iris-virginica
6   Iris-virginica   Iris-virginica
7   Iris-virginica   Iris-virginica
8   Iris-virginica   Iris-virginica
9   Iris-virginica   Iris-virginica
10  Iris-virginica   Iris-virginica
11  Iris-virginica   Iris-virginica
12  Iris-virginica   Iris-virginica
13  Iris-virginica   Iris-virginica
14  Iris-virginica   Iris-virginica
15  Iris-virginica   Iris-virginica
16  Iris-virginica   Iris-virginica
17  Iris-virginica   Iris-virginica
18  Iris-virginica   Iris-virginica
19  Iris-virginica   Iris-virginica
